In [1]:
try:
    import selenium
except ModuleNotFoundError:
    %pip install selenium

import numpy as np
import pandas as pd

from tqdm.notebook import tqdm
from time import sleep
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
def get_value(element):
    value = element.get_attribute("value")
    if value == "-1":
        return None
    return value

# Set up WebDriver
driver = webdriver.Chrome()

# Navigate to the website and manually solve the CAPTCHA
driver.get("https://airquality.cpcb.gov.in/ccr/#/caaqm-dashboard-all/caaqm-landing/caaqm-data-repository")

In [3]:
elements = driver.find_elements(By.CSS_SELECTOR, "ng-select")

# Select "Raw data" option
data_type = elements[0]
data_type.click()
data_type.find_elements(By.TAG_NAME, "li")[0].click()

# Select "15 min" frequency
frequency = elements[1]
frequency.click()
frequency.find_elements(By.TAG_NAME, "li")[0].click()

# Select state
state_element = elements[2]
state_element.click()
states = state_element.find_elements(By.TAG_NAME, "li")
for state in states:
    print("Scraping state:", state.text)
    
    state.click()
    
    # Select city
    city_element = elements[3]
    city_element.click()
    cities = city_element.find_elements(By.TAG_NAME, "li")
    
    for city in cities:
        print("Scraping city:", city.text)
        city.click()
        
        # Select station
        station_element = elements[4]
        station_element.click()
        
        stations = station_element.find_elements(By.TAG_NAME, "li")

        for station in stations:
            print("Scraping station:", station.text)
            station.click()
            
            # Search Submit button and press it
            buttons = driver.find_elements(By.TAG_NAME, "button")
            buttons[1].click()
            
            downloads = driver.find_elements(By.CLASS_NAME, "fa-download")
            for download in downloads:
                download.click()
                sleep(5)
            
            break
        break
    break

Scraping state: Andhra Pradesh
Scraping city: Amaravati
Scraping station: Secretariat, Amaravati - APPCB
